In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("engine_metadata.csv")
df

,CUSTOMER_NAME,PLANT_NAME,ENGINE_ID,FILE_ID
0,SPIFFY,SPIRITUAL-POLECAT,ENGINE_1,data_1.csv
1,SPIFFY,SPIRITUAL-POLECAT,ENGINE_2,data_2.csv
2,SPIFFY,SPIRITUAL-POLECAT,ENGINE_3,data_3.csv
3,NONCHALANT,NIFTY-ROOK,ENGINE_1,data_4.csv
4,NONCHALANT,NIFTY-ROOK,ENGINE_2,data_5.csv
...,...,...,...,...
207,SCEPTICAL,GIGANTIC-EARWIG,ENGINE_2,data_209.csv
208,MERCIFUL,PREMIUM-MOTH,ENGINE_1,data_210.csv
209,MERCIFUL,PREMIUM-MOTH,ENGINE_2,data_211.csv
210,MERCIFUL,PREMIUM-MOTH,ENGINE_3,data_212.csv


In [18]:
csv_per_engine = {}
grouped = df.groupby("ENGINE_ID")
for name, group in grouped:
    csv_per_engine[name] =  zip(group["FILE_ID"].values, group["CUSTOMER_NAME"].values)
    describe_data = []
    for csv_file, client_name in csv_per_engine[name]:
        describe_data.append(process_csv(csv_file))
    csv_per_engine[name] = zip(csv_per_engine[name], describe_data)


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/dataset/data_1.csv.csv'

In [ ]:
for data_path, client_name in csv_per_engine["ENGINE_1"]:
    print(data_path, client_name)

In [16]:
def add_working_hours(df):
    speed = df["CMP_SPEED"].values
    hours = [0]
    counter = 0
    for i in range(1, len(speed)):
        if speed[i-1] !=0 and speed[i] != 0:
            counter += 1
        hours.append(counter)
        # appending it to the df
        df["WORKING_HOURS"] = hours
    return df

In [17]:
def process_csv(file):
    df = pd.read_csv(f'dataset/dataset/{file}', encoding="ISO-8859-1")
    df = add_working_hours(df)
    df = df.dropna()
    return df.describe()